In [1]:
import os
import sys
import escher
import cobra
import numpy as np

sys.path.append("..")
from helpers.file_handling import  *

In [2]:
sbmls_path = "/mnt/d/data/GEMs/Com_8"
files = [os.path.join(sbmls_path, file) for file in os.listdir(sbmls_path) if file.endswith("ml") and "utf8" in file]
files

['/mnt/d/data/GEMs/Com_8/Bacteroides_thetaiotaomicron_dnLKV9_utf8.xml',
 '/mnt/d/data/GEMs/Com_8/Bacteroides_uniformis_dnLKV2_utf8.xml',
 '/mnt/d/data/GEMs/Com_8/Bacteroides_vulgatus_PC510_utf8.xml',
 '/mnt/d/data/GEMs/Com_8/Clostridium_ramosum_VPI_0427_DSM_1402_utf8.xml',
 '/mnt/d/data/GEMs/Com_8/Collinsella_aerofaciens_ERR1022282_utf8.xml',
 '/mnt/d/data/GEMs/Com_8/Fusobacterium_nucleatum_subsp_polymorphum_ATCC_10953_utf8.xml',
 '/mnt/d/data/GEMs/Com_8/Roseburia_intestinalis_XB6B4_utf8.xml',
 '/mnt/d/data/GEMs/Com_8/Ruminococcus_gnavus_ATCC_29149_utf8.xml']

In [3]:
"""
for file in files:
    convert_to_utf8(file)
"""

'\nfor file in files:\n    convert_to_utf8(file)\n'

In [15]:
sb_model = cobra.io.read_sbml_model(files[3])
sb_model

Name,M_Clostridium_ramosum_VPI_0427_DSM_1402
Memory address,7efe148a6980
Number of metabolites,962
Number of reactions,1049
Number of genes,893
Number of groups,68
Objective expression,1.0*biomass525 - 1.0*biomass525_reverse_5c178
Compartments,"Cytoplasm, Extracellular"


In [16]:
solution = sb_model.optimize()
solution

,fluxes,reduced_costs
12aHSDHe,615.247537,2.662693e-17
23DHMPO,0.000000,-7.528465e-18
26DAPLLATi,20.234369,6.071532e-18
2AHBUTI,0.000000,-2.174890e-17
2MBCOATA,0.000000,0.000000e+00
...,...,...
tCAP,0.000000,-0.000000e+00
tCZP,-243.058162,-2.738546e-19
tNCAP,0.000000,1.041099e-17
tNZP,0.000000,-2.738546e-19


In [6]:
def search_metab_by_name(model, met_name:str):
    hits = []
    for met in model.metabolites:
        if met_name in met.name:
            hits.append(met)
    return hits

def search_metab_by_weight(model, weight:float, tolerance:float):
    hits = []
    for met in model.metabolites:
        if "R" in met.formula or "X" in met.formula:
            continue
        if np.isclose(weight, met.formula_weight, atol=tolerance, rtol=0.0):
            hits.append(met)
    return hits

In [22]:
m_hits = search_metab_by_name(sb_model, "2-C-methyl-D-erythritol 4-phosphate")

In [23]:
m_hits[0]

Metabolite identifier,2me4p[c]
Name,2-C-methyl-D-erythritol 4-phosphate
Memory address,0x7efe023f65c0
Formula,C5H11O7P
Compartment,c
In 2 reaction(s),"MEPCT, DXPRIr"


In [8]:
search_metab_by_weight(sb_model, 983.8475, 2e-4)

[<Metabolite 10fthf5glu[c] at 0x7efe152ad0f0>]

In [9]:
sb_model_json = cobra.io.to_json(sb_model)

In [27]:
builder = escher.Builder(model_json=sb_model_json)
builder.model = sb_model
builder.reaction_data = solution.fluxes
builder.metabolite_data = solution.shadow_prices
builder.scroll_behavior = 'zoom'
builder.reaction_styles = ['size']

In [28]:
builder

Builder(reaction_data={'12aHSDHe': 615.2475366688848, '23DHMPO': 0.0, '26DAPLLATi': 20.23436890252726, '2AHBUT…

In [29]:
builder.save_html('../../reports/pathway/example_map.html')